In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import os 

# CHARGEMENT DES DONNEES 

Ajouter dans les données les grèves et données métérologique

In [2]:
dialect = 'mysql+pymysql'
user = 'root'
password = os.getenv('mdp_mySQL')
host = 'localhost'
port = 3306
database_idf = 'SNCF_TGV_TER'

engine_url = f"{dialect}://{user}:{password}@{host}:{port}/{database_idf}"
engine = create_engine(engine_url)

In [3]:
df_disruption = pd.read_sql('disruptions_tgv_ter',engine)
df_disruption

,id_disruption,vehicle_id,train_type,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44199,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,TGV,stop_point:SNCF:85010082:LongDistanceTrain,Genève Cornavin,6.142440,46.2102,2024-03-25 16:17:00,2024-03-25 20:29:00,15.0,15.0,Incident de circulation,2024-03-25
44200,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,TGV,stop_point:SNCF:85011205:LongDistanceTrain,Lausanne,6.629090,46.5168,2024-03-25 16:17:00,2024-03-25 20:29:00,15.0,15.0,Incident de circulation,2024-03-25
44201,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,TGV,stop_point:SNCF:87686006:LongDistanceTrain,Paris - Gare de Lyon - Hall 1 & 2,2.373480,48.8449,2024-03-25 16:17:00,2024-03-25 20:29:00,0.0,0.0,Incident de circulation,2024-03-25
44202,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,TGV,stop_point:SNCF:87743005:LongDistanceTrain,Bourg-en-Bresse,5.215010,46.2001,2024-03-25 16:17:00,2024-03-25 20:29:00,15.0,15.0,Incident de circulation,2024-03-25


In [4]:
df_vehicle = pd.read_sql('vehicle_journeys_tgv_ter', engine)
df_vehicle['vehicle_id'] = df_vehicle['vehicle_id'].apply(lambda x: ':'.join(x.split(':')[1:]))
df_vehicle

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
0,SNCF:2024-03-18:117752:1187:LongDistanceTrain,route:SNCF:FR:Line::B10C45A0-C32C-4232-85F2-4B...,05:00:00,07:19:00,Train grande vitesse,0f4e5f2c-a148-438f-b7cb-71b80199e8d4,2024-03-18
1,SNCF:2024-03-18:12244:1187:LongDistanceTrain,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,300d66cc-2614-4087-89d7-89abe997e2cf,2024-03-18
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
3,SNCF:2024-03-18:12254:1187:LongDistanceTrain,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
64088,SNCF:2024-03-25:9896:1187:LongDistanceTrain,route:SNCF:CSR:071100,15:56:00,23:49:00,Train grande vitesse,None,2024-03-25
64089,SNCF:2024-03-25:9898:1187:LongDistanceTrain,route:SNCF:CSR:071200,06:31:00,14:15:00,Train grande vitesse,None,2024-03-25
64090,SNCF:2024-03-25:9916:0018:LongDistanceTrain,route:SNCF:0018-84000588-87111849,07:11:00,11:05:00,Train grande vitesse,None,2024-03-25
64091,SNCF:2024-03-25:99209:1187:LongDistanceTrain,route:SNCF:FR:Line::bd8f7fb5-658e-4960-a87d-de...,17:44:00,23:10:00,Train grande vitesse,524128f9-4310-414a-9276-086a08a3416e,2024-03-25


In [5]:
df_lines = pd.read_sql_table('pt_lines_tgv_ter', engine)
df_lines

,route_id,route_name,train_type,network_name,opening_time,closing_time
0,route:SNCF:0018-80100537-87271007,Dortmund Hbf - Paris Nord,Train grande vitesse,Eurostar,05:18:00,12:21:00
1,route:SNCF:0018-80101840-87271007,Essen Hbf - Paris Nord,Train grande vitesse,Eurostar,07:35:00,22:14:00
2,route:SNCF:0018-80154583-87271007,Köln - Hauptbahnhof - Paris Nord,Train grande vitesse,Eurostar,05:21:00,23:14:00
3,route:SNCF:0018-84000588-87111849,Amsterdam Centraal - Marne-la-Vallée - Chessy,Train grande vitesse,Eurostar,06:47:00,11:05:00
4,route:SNCF:0018-84000588-87271007,Amsterdam Centraal - Paris Nord,Train grande vitesse,Eurostar,06:11:00,23:43:00
...,...,...,...,...,...,...
363,route:stop_area:SNCF:87113001_stop_area:SNCF:8...,Paris Est - Strasbourg,Train grande vitesse,additional service,None,None
364,route:stop_area:SNCF:87394007_stop_area:SNCF:8...,Chartres - Versailles Chantiers,Train grande vitesse,additional service,None,None
365,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Paris Saint-Lazare,Train grande vitesse,additional service,None,None
366,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Le Havre,Train grande vitesse,additional service,None,None


# PREPARATION DES DONNEES 

## Merge des tables 

Probleme que je ne comprend pas encore : il y a beaucoup plus de perturbation dans df_vehicle que dans df_disruption

In [6]:
df_merge_inter = pd.merge(df_disruption.drop(['data_date','train_type'],axis=1), df_vehicle, on=['vehicle_id','id_disruption'], how='left')
df_merge = pd.merge(df_merge_inter, df_lines.drop('train_type', axis=1), on='route_id', how='left')

In [7]:
df_merge_clean = df_merge.dropna(subset=['arrival_delay', 'departure_delay'])
df_merge_clean

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date,route_name,network_name,opening_time,closing_time
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44199,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,stop_point:SNCF:85010082:LongDistanceTrain,Genève Cornavin,6.142440,46.2102,2024-03-25 16:17:00,2024-03-25 20:29:00,15.0,15.0,Incident de circulation,route:SNCF:CSR:622500,16:17:00,20:14:00,Train grande vitesse,2024-03-25,Paris - Gare de Lyon - Hall 1 & 2 - Genève Cor...,TGV Lyria,06:13:00,23:42:00
44200,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,stop_point:SNCF:85011205:LongDistanceTrain,Lausanne,6.629090,46.5168,2024-03-25 16:17:00,2024-03-25 20:29:00,15.0,15.0,Incident de circulation,route:SNCF:CSR:622500,16:17:00,20:14:00,Train grande vitesse,2024-03-25,Paris - Gare de Lyon - Hall 1 & 2 - Genève Cor...,TGV Lyria,06:13:00,23:42:00
44201,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,stop_point:SNCF:87686006:LongDistanceTrain,Paris - Gare de Lyon - Hall 1 & 2,2.373480,48.8449,2024-03-25 16:17:00,2024-03-25 20:29:00,0.0,0.0,Incident de circulation,route:SNCF:CSR:622500,16:17:00,20:14:00,Train grande vitesse,2024-03-25,Paris - Gare de Lyon - Hall 1 & 2 - Genève Cor...,TGV Lyria,06:13:00,23:42:00
44202,93ef55e9-9b47-41f7-8245-5f74a7096f95,SNCF:2024-03-25:9777:1187:LongDistanceTrain,stop_point:SNCF:87743005:LongDistanceTrain,Bourg-en-Bresse,5.215010,46.2001,2024-03-25 16:17:00,2024-03-25 20:29:00,15.0,15.0,Incident de circulation,route:SNCF:CSR:622500,16:17:00,20:14:00,Train grande vitesse,2024-03-25,Paris - Gare de Lyon - Hall 1 & 2 - Genève Cor...,TGV Lyria,06:13:00,23:42:00


### Rajout des trajets non perturbé

In [8]:
df_vehicle_add = df_vehicle[df_vehicle['id_disruption'].isna()]
df_vehicle_add

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
6,SNCF:2024-03-18:12259:1187:LongDistanceTrain,route:SNCF:CSR:421400,12:04:00,14:14:00,Train grande vitesse,None,2024-03-18
7,SNCF:2024-03-18:12260:1187:LongDistanceTrain,route:SNCF:CSR:421400,14:46:00,17:03:00,Train grande vitesse,None,2024-03-18
9,SNCF:2024-03-18:12265:1187:LongDistanceTrain,route:SNCF:CSR:421400,13:58:00,16:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
64087,SNCF:2024-03-25:9894:1187:LongDistanceTrain,route:SNCF:CSR:005400,17:01:00,21:01:00,Train grande vitesse,None,2024-03-25
64088,SNCF:2024-03-25:9896:1187:LongDistanceTrain,route:SNCF:CSR:071100,15:56:00,23:49:00,Train grande vitesse,None,2024-03-25
64089,SNCF:2024-03-25:9898:1187:LongDistanceTrain,route:SNCF:CSR:071200,06:31:00,14:15:00,Train grande vitesse,None,2024-03-25
64090,SNCF:2024-03-25:9916:0018:LongDistanceTrain,route:SNCF:0018-84000588-87111849,07:11:00,11:05:00,Train grande vitesse,None,2024-03-25


In [9]:
#df_vehicle_add_merge  = pd.merge(df_vehicle_add,df_lines.drop('train_type', axis=1), on='route_id', how='left')
#df_vehicle_add_merge

In [10]:
df_vehicle_add.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55684 entries, 2 to 64092
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   vehicle_id     55684 non-null  object        
 1   route_id       55684 non-null  object        
 2   time_begin     55684 non-null  object        
 3   time_end       55684 non-null  object        
 4   train_type     55684 non-null  object        
 5   id_disruption  0 non-null      object        
 6   data_date      55684 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 3.4+ MB


In [11]:
df_final = pd.concat([df_merge_inter, df_vehicle_add], ignore_index=True)
df_final

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,None,SNCF:2024-03-25:9894:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:005400,17:01:00,21:01:00,Train grande vitesse,2024-03-25
99884,None,SNCF:2024-03-25:9896:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:071100,15:56:00,23:49:00,Train grande vitesse,2024-03-25
99885,None,SNCF:2024-03-25:9898:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:071200,06:31:00,14:15:00,Train grande vitesse,2024-03-25
99886,None,SNCF:2024-03-25:9916:0018:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:0018-84000588-87111849,07:11:00,11:05:00,Train grande vitesse,2024-03-25


## Choix des variables

In [12]:
df_final['disrupted'] = np.where(df_final['id_disruption'].isna(), 0, 1)

In [13]:
df_final['week_day'] = df_final['data_date'].dt.weekday

In [14]:
df_final

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date,disrupted,week_day
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,1,0
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,1,0
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,1,0
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,1,0
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,None,SNCF:2024-03-25:9894:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:005400,17:01:00,21:01:00,Train grande vitesse,2024-03-25,0,0
99884,None,SNCF:2024-03-25:9896:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:071100,15:56:00,23:49:00,Train grande vitesse,2024-03-25,0,0
99885,None,SNCF:2024-03-25:9898:1187:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:CSR:071200,06:31:00,14:15:00,Train grande vitesse,2024-03-25,0,0
99886,None,SNCF:2024-03-25:9916:0018:LongDistanceTrain,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:0018-84000588-87111849,07:11:00,11:05:00,Train grande vitesse,2024-03-25,0,0


In [15]:
df_final_v2 = df_final[[#'id_disruption', 
        #'vehicle_id', 
        'id_stop', 
        #'name_stop', 
        #'lon', 
        #'lat',
        'disruption_start',
        'disruption_end', 
        'arrival_delay',
        'departure_delay', 
        'cause_delay',
        'route_id',
        'time_begin',
        'time_end',
        'train_type',
        #'data_date',
        'disrupted',
        'week_day']]

In [16]:
# Création de valeurs binaires pour chaque type de train et jour de la semaine
df_final_v2 = pd.get_dummies(df_final_v2, columns=['train_type','week_day'])

In [17]:
# On créer des tranches d'heures puis indique si le train a circulé dans cette tranche
tranches = list(range(4*60, 25*60, 60))

def train_dans_tranche(depart, arrivee, tranche_debut, tranche_fin):
    return not (arrivee <= tranche_debut or depart >= tranche_fin)

for i in range(len(tranches)-1):
    tranche_nom = f'{tranches[i]//60:02d}h-{tranches[i+1]//60:02d}h'
    df_final_v2[tranche_nom] = df_final_v2.apply(lambda x: train_dans_tranche(x['time_begin'].hour*60 + x['time_begin'].minute, x['time_end'].hour*60 + x['time_end'].minute, tranches[i], tranches[i+1]), axis=1)


In [18]:
# On remplace les trajets et les gares par leur fréquence respective
route_freq = df_final_v2['route_id'].value_counts() / len(df_final_v2)
df_final_v2['route_freq'] = df_final_v2['route_id'].map(route_freq)

stop_freq = df_final_v2['id_stop'].value_counts() / len(df_final_v2[~df_final_v2['id_stop'].isna()])
df_final_v2['stop_freq'] = df_final_v2['id_stop'].map(stop_freq)

In [19]:
# Attribution d'un nombre à chaque cause de perturbation
df_final_v2['cause_delay_label'] = df_final_v2['cause_delay']
df_final_v2['cause_delay'] = pd.factorize(df_final_v2['cause_delay'])[0]

In [20]:
df_final_v2 = df_final_v2.drop(['time_begin', 'time_end','route_id', 'id_stop'], axis=1)

In [21]:
df_final_v2

,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,disrupted,train_type_TER / Intercités,train_type_Train grande vitesse,week_day_0,week_day_1,...,17h-18h,18h-19h,19h-20h,20h-21h,21h-22h,22h-23h,23h-24h,route_freq,stop_freq,cause_delay_label
0,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,0,1,False,True,True,False,...,False,True,True,True,False,False,False,0.003524,0.007217,Défaut d'alimentation électrique
1,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,0,1,False,True,True,False,...,False,True,True,True,False,False,False,0.003524,0.001312,Défaut d'alimentation électrique
2,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,0,1,False,True,True,False,...,False,True,True,True,False,False,False,0.003524,0.004072,Défaut d'alimentation électrique
3,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,1,1,False,True,True,False,...,False,False,False,False,False,False,False,0.003524,0.007217,Régulation du trafic
4,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,1,1,False,True,True,False,...,False,False,False,False,False,False,False,0.003524,0.004072,Régulation du trafic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,NaT,NaT,NaN,NaN,-1,0,False,True,True,False,...,True,True,True,True,True,False,False,0.000501,NaN,NaN
99884,NaT,NaT,NaN,NaN,-1,0,False,True,True,False,...,True,True,True,True,True,True,True,0.001392,NaN,NaN
99885,NaT,NaT,NaN,NaN,-1,0,False,True,True,False,...,False,False,False,False,False,False,False,0.001251,NaN,NaN
99886,NaT,NaT,NaN,NaN,-1,0,False,True,True,False,...,False,False,False,False,False,False,False,0.000050,NaN,NaN


# 

# 

# Trajet perturbé ou non

### RANDOM FOREST (CLASSIFICATION)

In [65]:
df_rf_cla = df_final_v2.drop(['stop_freq',
                'disruption_start',
                'disruption_end',
                'arrival_delay',
                'departure_delay',
                'cause_delay',
                'train_type_Train grande vitesse',
                'train_type_TER / Intercités',
                'cause_delay_label'],
                             axis=1)

In [66]:
df_rf_cla

,disrupted,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,04h-05h,05h-06h,...,15h-16h,16h-17h,17h-18h,18h-19h,19h-20h,20h-21h,21h-22h,22h-23h,23h-24h,route_freq
0,1,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,0.003524
1,1,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,0.003524
2,1,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,0.003524
3,1,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.003524
4,1,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.003524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,0,True,False,False,False,False,False,False,False,False,...,False,False,True,True,True,True,True,False,False,0.000501
99884,0,True,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,0.001392
99885,0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.001251
99886,0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.000050


In [67]:
X_rf_cla = df_rf_cla.drop('disrupted', axis=1)
y_rf_cla = df_rf_cla['disrupted']

X_train_rf_cla, X_test_rf_cla, y_train_rf_cla, y_test_rf_cla = train_test_split(X_rf_cla, y_rf_cla, test_size=0.2, random_state=10)

In [68]:
rf = RandomForestClassifier(random_state=10)
rf.fit(X_train_rf_cla, y_train_rf_cla)

RandomForestClassifier(random_state=10)

In [69]:
# Prédictions
predictions = rf.predict(X_test_rf_cla)

# Évaluation
print(classification_report(y_test_rf_cla, predictions))
print("ROC AUC score:", roc_auc_score(y_test_rf_cla, predictions))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91     16688
           1       0.84      0.97      0.90     13279

    accuracy                           0.91     29967
   macro avg       0.91      0.91      0.91     29967
weighted avg       0.92      0.91      0.91     29967

ROC AUC score: 0.9144975401438714


###  Ajustements des paramètres

In [ ]:
param_grid = {
    'n_estimators': [200,300,400],  
    'max_depth': [None, 10,30,50],  
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_rf_cla, y_train_rf_cla)

print("Meilleurs paramètres:", grid_search.best_params_)

In [ ]:
rf_best = RandomForestClassifier(**grid_search.best_params_,random_state=10)
rf_best.fit(X_train_rf_cla, y_train_rf_cla)

In [ ]:
# Prédictions
predictions_best = rf_best.predict(X_test_rf_cla)

# Évaluation
print(classification_report(y_test_rf_cla, predictions_best))
print("ROC AUC score:", roc_auc_score(y_test_rf_cla, predictions_best))

In [ ]:
importances = rf_best.feature_importances_

features = X_train_rf_cla.columns
importance_scores = sorted(list(zip(features, importances)), key=lambda x: x[1], reverse=True)

for feature, importance in importance_scores:
    print(f"{feature}: {importance}")

séparer semaine et weekend 

# 

# Cause de perturbation

### RANDOM FOREST (CLASSIFICATION)

In [27]:
df_rf_cla2 = df_final_v2[df_final_v2['disrupted']==1] 
df_rf_cla2 = df_rf_cla2[df_rf_cla2['cause_delay']>=0] 
df_rf_cla2 = df_rf_cla2.drop(['disruption_start', 
                       'disruption_end', 
                       'arrival_delay',
                        'departure_delay', 
                       'disrupted',
                        'train_type_TER / Intercités',
                        'train_type_Train grande vitesse',
                        'stop_freq',
                        'cause_delay_label'],
                        axis=1)

In [28]:
df_rf_cla2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41716 entries, 0 to 44203
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cause_delay  41716 non-null  int64  
 1   week_day_0   41716 non-null  bool   
 2   week_day_1   41716 non-null  bool   
 3   week_day_2   41716 non-null  bool   
 4   week_day_3   41716 non-null  bool   
 5   week_day_4   41716 non-null  bool   
 6   week_day_5   41716 non-null  bool   
 7   week_day_6   41716 non-null  bool   
 8   04h-05h      41716 non-null  bool   
 9   05h-06h      41716 non-null  bool   
 10  06h-07h      41716 non-null  bool   
 11  07h-08h      41716 non-null  bool   
 12  08h-09h      41716 non-null  bool   
 13  09h-10h      41716 non-null  bool   
 14  10h-11h      41716 non-null  bool   
 15  11h-12h      41716 non-null  bool   
 16  12h-13h      41716 non-null  bool   
 17  13h-14h      41716 non-null  bool   
 18  14h-15h      41716 non-null  bool   
 19  15h-16h  

In [62]:
X_rf_cla2 = df_rf_cla2.drop('cause_delay', axis=1)
y_rf_cla2 = df_rf_cla2['cause_delay']

X_train_rf_cla2, X_test_rf_cla2, y_train_rf_cla2, y_test_rf_cla2 = train_test_split(X_rf_cla2, y_rf_cla2, test_size=0.2, random_state=42)

In [63]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_rf_cla2, y_train_rf_cla2)

RandomForestClassifier(random_state=42)

In [64]:
# Faire des prédictions sur l'ensemble de test
predictions = rf.predict(X_test_rf_cla2)

# Évaluation des prédictions
print(classification_report(y_test_rf_cla2, predictions))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89        86
           1       0.94      0.94      0.94      2101
           2       0.96      0.94      0.95       518
           3       0.91      0.91      0.91      1593
           4       0.89      0.87      0.88       202
           5       0.97      0.97      0.97       258
           6       1.00      1.00      1.00         4
           7       0.91      0.95      0.93       550
           8       0.93      0.91      0.92       453
           9       0.95      0.94      0.94       386
          10       0.96      0.97      0.96        68
          11       0.90      0.90      0.90        72
          12       0.95      0.93      0.94        42
          13       0.98      0.86      0.92       109
          14       1.00      0.90      0.95       108
          15       0.98      0.92      0.95       137
          16       0.94      0.98      0.96        65
          17       0.91    

# 

# Quantification du retard

### RANDOM FOREST (REGRESSION)

In [51]:
df_rf_reg = df_final_v2[df_final_v2['disrupted']==1] 
df_rf_reg = df_rf_reg[df_rf_reg['cause_delay']>=0] 
df_rf_reg = df_rf_reg.drop(['disruption_start', 
                       'disruption_end', 
                       'arrival_delay', 
                       'disrupted',
                        'cause_delay_label'],
                        axis=1)

In [52]:
cause_freq = df_rf_reg['cause_delay'].value_counts() / len(df_rf_reg)
df_rf_reg['cause_freq'] = df_rf_reg['cause_delay'].map(cause_freq)
df_rf_reg = df_rf_reg.drop('cause_delay', axis=1)

In [53]:
df_rf_reg = df_rf_reg[~df_rf_reg['departure_delay'].isna()]

In [54]:
df_rf_reg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40354 entries, 0 to 44203
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   departure_delay                  40354 non-null  float64
 1   train_type_TER / Intercités      40354 non-null  bool   
 2   train_type_Train grande vitesse  40354 non-null  bool   
 3   week_day_0                       40354 non-null  bool   
 4   week_day_1                       40354 non-null  bool   
 5   week_day_2                       40354 non-null  bool   
 6   week_day_3                       40354 non-null  bool   
 7   week_day_4                       40354 non-null  bool   
 8   week_day_5                       40354 non-null  bool   
 9   week_day_6                       40354 non-null  bool   
 10  04h-05h                          40354 non-null  bool   
 11  05h-06h                          40354 non-null  bool   
 12  06h-07h                

In [59]:
X_rf_reg = df_rf_reg.drop('departure_delay', axis=1)
y_rf_reg = df_rf_reg['departure_delay']
X_train_rf_reg, X_test_rf_reg, y_train_rf_reg, y_test_rf_reg = train_test_split(X_rf_reg, y_rf_reg, test_size=0.2, random_state=42)

In [60]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train_rf_reg, y_train_rf_reg)

RandomForestRegressor(random_state=42)

In [61]:
# Prédictions de retard en minutes
predictions_reg = rf.predict(X_test_rf_reg)

# Évaluation
print("MSE:", mean_squared_error(y_test_rf_reg, predictions_reg))
print("MAE:", mean_absolute_error(y_test_rf_reg, predictions_reg))
print("R² score:", r2_score(y_test_rf_reg, predictions_reg))

MSE: 2751.1626452338346
MAE: 5.71993418841891
R² score: 0.11293890103890347


In [58]:
importances = rf.feature_importances_

features = X_train_rf_reg.columns
importance_scores = sorted(list(zip(features, importances)), key=lambda x: x[1], reverse=True)

for feature, importance in importance_scores:
    print(f"{feature}: {importance}")

stop_freq: 0.47600595081572394
route_freq: 0.18055155240849155
cause_freq: 0.08862842165471406
22h-23h: 0.031136436463877242
week_day_4: 0.02643468592032474
23h-24h: 0.02268188622493276
week_day_0: 0.022067619973403608
19h-20h: 0.018370654165449696
17h-18h: 0.01612333328877916
18h-19h: 0.013464315135510703
week_day_5: 0.012400335261164967
week_day_6: 0.012205015059607853
week_day_3: 0.010680103760121348
train_type_Train grande vitesse: 0.008758976073243715
21h-22h: 0.00839537030087025
train_type_TER / Intercités: 0.008267438086545504
week_day_1: 0.005601073012861153
08h-09h: 0.005078642143083174
20h-21h: 0.005022740979073919
week_day_2: 0.004568562172065507
09h-10h: 0.004385108225690338
16h-17h: 0.003983441122047448
06h-07h: 0.0023930282206189382
12h-13h: 0.0020951816089723315
13h-14h: 0.001895480562848064
05h-06h: 0.0017336073682402181
14h-15h: 0.0016540006017489945
15h-16h: 0.0015728848757308688
11h-12h: 0.0015172850467447025
10h-11h: 0.001262237248798858
07h-08h: 0.00101138932568349